# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение, количество детей, уровень дохода клиента и цель кредита на факт его погашения в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

# Описание данных

* `children` — количество детей в семье
* `days_employed` — общий трудовой стаж в днях
* `dob_years` — возраст клиента в годах
* `education` — уровень образования клиента
* `education_id` — идентификатор уровня образования
* `family_status` — семейное положение
* `family_status_id` — идентификатор семейного положения
* `gender` — пол клиента
* `income_type` — тип занятости
* `debt` — имел ли задолженность по возврату кредитов
* `total_income` — ежемесячный доход
* `purpose` — цель получения кредита

## Загрузка и обзор данных

In [225]:
import pandas as pd

In [226]:
data = pd.read_csv('/datasets/data.csv')

data.info()
display(data.head(10)) # получим общую информацию


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [227]:
data.dtypes # посмотрим типы данных

children              int64
days_employed       float64
dob_years             int64
education            object
education_id          int64
family_status        object
family_status_id      int64
gender               object
income_type          object
debt                  int64
total_income        float64
purpose              object
dtype: object

In [228]:
data.describe() # посмотрим статистическую информацию

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


**Вывод**

В таблице 12 столбцов, нарушений в стиле и синтаксисе заголовков нет. В числовых данных видны следующие аномалии, дубликаты и пропущенные значения:

- пропуски в `days_employed` и `total_income` 
- аномальные значения количества детей (-1 и 20) в `children`
- отрицательное значение дней трудового стажа в `days_employed`
- вместо возраста в `dob_years` в некотрых строках 0
- разный регистр букв в столбце `education`
- разные формы записи одной и той же цели кредита в столбце `purpose`

Также в предобработке необходимо проверить колонки типа object на предмет проблем с данными

## Предобработка данных

### Обработка пропусков

In [229]:
data.isna().sum() # количество пропущенных значений

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [230]:
# посмотрим строки, где пропущены значения одновременно в двух столбцах

data[(data['days_employed'].isna()) & (data['total_income'].isna())]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Среднее,1,женат / замужем,0,M,компаньон,0,NaN,сделка с автомобилем
21495,1,NaN,50,среднее,1,гражданский брак,1,F,сотрудник,0,NaN,свадьба
21497,0,NaN,48,ВЫСШЕЕ,0,женат / замужем,0,F,компаньон,0,NaN,строительство недвижимости
21502,1,NaN,42,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,строительство жилой недвижимости


Пропуски в столбцах `days_employed` и `total_income` совпадают для каждого заемщика, возможно, эта ошибка произошла при выгрузке, либо данные не были внесены изначально. Это могут быть зарплатные клиенты или просто те, кто давно пользуются услугами данного банка: в системе есть информация о движениях средств на их счете, для них уже готово одобренное предложения с пониженной процентной ставкой и подтверждение дохода не требуется. Либо не требуется оно, если заявка на кредит ниже определенной суммы - во многих банках довольно лояльные условия и можно получить небольшой займ без справки о доходах.

Пропуски составляют примерно 1/10 всей таблицы, поэтому при удалении мы потеряем большое количество данных в других нужных нам колонках. Пропущенные значения в графе дохода можно заполнить медианными по типу занятости.

In [231]:
# группируем по типу занятости, находим характерное значение для каждой группы, заполняем с помощью лямбда-функции

data['total_income'] = data.groupby(
    'income_type')['total_income'].apply(lambda x: x.fillna(x.median()))

data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income           0
purpose                0
dtype: int64

In [232]:
# конвертируем отрицательные значения стажа в положительные

data['days_employed'] = data['days_employed'].fillna(0).abs()

In [233]:
# посмотрим статистику трудового стажа по типу занятости

data.groupby('income_type')['days_employed'].describe()


,count,mean,std,min,25%,50%,75%,max
income_type,,,,,,,,
безработный,2.0,366413.652744,40855.478519,337524.466835,351969.059790,366413.652744,380858.245699,395302.838654
в декрете,1.0,3296.759962,NaN,3296.759962,3296.759962,3296.759962,3296.759962,3296.759962
госслужащий,1459.0,3057.343890,2835.336816,0.000000,854.364241,2385.358043,4454.221934,15193.032201
компаньон,5085.0,1900.579581,2043.979915,0.000000,451.152387,1311.128244,2672.323106,17615.563266
пенсионер,3856.0,325909.496980,114633.231469,0.000000,340199.573133,360505.668544,380751.301140,401755.400475
предприниматель,2.0,260.424042,368.295212,0.000000,130.212021,260.424042,390.636063,520.848083
сотрудник,11119.0,2095.293025,2298.175038,0.000000,505.391128,1360.363902,2895.108262,18388.949901
студент,1.0,578.751554,NaN,578.751554,578.751554,578.751554,578.751554,578.751554


Интересно, что предпренимателей, студентов, безработных и декретников считанные единицы - видимо банк не выдает кредиты лицам без фиксированного дохода. При этом компаньонов достаточно - возможно это созаемщики или предаставляющие справку о доходах супруга. Помимо пропусков в колонке `days_employed` отрицательные значения и неправдоподобно большие у пенсионеров и безработных. Вероятно, это некорректный расчет количества дней по датам приема/увольнения. Вообще для одобрения заявки трудовой стаж не нужен, за исключением ипотечных кредитов. В этом случае запрос отправляется в пенсионный фонд, поэтому возможны ошибки или другой формат записи. 

Данные об общем стаже никак не фигурируют в тех задании и не влияют на исследование - как видно из статистического описания, кредиты выдавались и тем, чье количество трудовых дней равно нулю (видимо информация не вносилась за ненадобностью) Поэтому эту колонку в данном случае допустимо удалить. В ином варианте стоит обратиться к заказчику за уточнением и с указанием проблемы в этих данных.

In [234]:
# удаляем столбец с трудовым стажем 

data = data.drop(columns='days_employed') 

### Замена типа данных

Значения в колонке `total_income` имеют тип данных float, для расчетов нам не нужно такое количество знаков после точки, так как копейки не играют никакой роли, поэтому приведем данные к целочисленному типу int.

In [235]:
data['total_income'] = data['total_income'].astype('int') 

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21525 non-null  int64 
 1   dob_years         21525 non-null  int64 
 2   education         21525 non-null  object
 3   education_id      21525 non-null  int64 
 4   family_status     21525 non-null  object
 5   family_status_id  21525 non-null  int64 
 6   gender            21525 non-null  object
 7   income_type       21525 non-null  object
 8   debt              21525 non-null  int64 
 9   total_income      21525 non-null  int64 
 10  purpose           21525 non-null  object
dtypes: int64(6), object(5)
memory usage: 1.8+ MB


### Обработка дубликатов


In [236]:
print(data.duplicated().sum()) # посчитаем явные дубликаты в таблице.


54


Количество строк-дубликатов в таблице - 54, это примерно 0,2%

Есть вероятность, что это не дубликаты, так как уникальные значения в столбце ежемесячного дохода, которые могли бы точно указать дубликаты, заменены при обработке пропусков медианой. Это вполне могут быть разные заемщики с одинаковым набором распрорстраненных данных - пол, статус, дети и т.д. Поэтому эти строки мы удалять не будем. А в случае если это все же дубликаты, 0,2% не повлиют на итоги.

Далее найдем уникальные значения столбцов `education` и `family_status`, так как при получении общей информации о таблице видно, что данные внесены с использованием прописных и строчных букв.
Приведем символы к нижнему регистру, чтобы избавиться от дубликатов, а в случае с колонкой `family_status` - чтобы исключить ошибку при обращении к ячейке датафрема.

In [237]:
print(data['education'].sort_values().unique()) # получаем уникальные значения

print(data['family_status'].sort_values().unique())


['ВЫСШЕЕ' 'Высшее' 'НАЧАЛЬНОЕ' 'НЕОКОНЧЕННОЕ ВЫСШЕЕ' 'Начальное'
 'Неоконченное высшее' 'СРЕДНЕЕ' 'Среднее' 'УЧЕНАЯ СТЕПЕНЬ'
 'Ученая степень' 'высшее' 'начальное' 'неоконченное высшее' 'среднее'
 'ученая степень']
['Не женат / не замужем' 'в разводе' 'вдовец / вдова' 'гражданский брак'
 'женат / замужем']


In [238]:
data['education'] = data['education'].str.lower() # приводим к нижнему регистру

data['family_status'] = data['family_status'].str.lower()

Проверим, что все прописные символы в столбцах переведены в строчные.

In [239]:
print(data['education'].sort_values().unique())

print(data['family_status'].sort_values().unique())

['высшее' 'начальное' 'неоконченное высшее' 'среднее' 'ученая степень']
['в разводе' 'вдовец / вдова' 'гражданский брак' 'женат / замужем'
 'не женат / не замужем']


### Другие проблемы в данных

Были выявлены другие проблемы в записи данных: в столбце `children` есть нереалистичные показатели - отрицательное значение "-1" и очень высокий показатель - 20 детей.

In [240]:
data['children'].value_counts() # посмотрим количество уникальных значений

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Отрицательное значение вероятно появилось при вводе данных в разном формате: в бинарном виде -1 означает 0 или "нет". Очевидно таким образом были маркированы клиенты без детей, а всех с детьми маркировали как 1. Двадцать детей можно было бы допустить у одного-двух заемщиков, но не у 76, к тому же в данных присутствовали бы другие многодетные семьи, например 10 или 15 детей. 
Поэтому 20 - скорей всего ошибка при вводе, клавиши 2 и 0 на numlock соседние, поэтому приведем к наиболее вероятному варианту - "2"

In [255]:
data['children'] = data['children'].replace(-1, 0) # заменяем значения

data['children'] = data['children'].replace(20, 2)

In [256]:
data['children'].value_counts() # проверяем результат

0    14196
1     4818
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

При обзоре данных было также обнаружено, что вместо возраста у некотрых клиентов стоит 0, видимо таким образом заполнили отсутствующие данные. Зависимости возврата кредита в срок от возраста в запросах нет, но в случае если в дальнейшем придется обратиться к этой колонке, заменим на 'unknown' во избежание ошибок

In [243]:
data['dob_years'] = data['dob_years'].replace(0, 'unknown') # заменяем нулевой возраст на неизвестный

### Лемматизация

Получим список уникальных значений столбца `purpose`. Одни и те же цели получения кредита записаны в разных выражениях, чтобы в дальнейшем сгруппировать заемщиков по данному призанку, необходимо привести лемматизацию.


In [244]:
data['purpose'].value_counts() # посчитаем количество целей

свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
операции с жильем                         653
покупка жилья для сдачи                   653
операции с коммерческой недвижимостью     651
жилье                                     647
покупка жилья                             647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
строительство недвижимости                620
покупка своего жилья                      620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

In [245]:
from pymystem3 import Mystem # загружаем модуль
m = Mystem() 

data['purpose_lemm'] = data['purpose'].apply(m.lemmatize) # добавим столбец с леммами

data['purpose_lemm']

def purpose_group(purpose_lemm):
    if 'образование'in purpose_lemm:
        return 'образование'
    if 'свадьба'in purpose_lemm:
        return 'свадьба'
    if 'автомобиль'in purpose_lemm:
        return 'автомобиль'
    if 'жилье' or 'недвижимость'in purpose_lemm:
        return 'недвижимость'


data['purpose_group'] = data['purpose_lemm'].apply(purpose_group) # добавляем столбец с целями по группам


In [246]:
data['purpose_group'].value_counts() # проверяем результат


недвижимость    10840
автомобиль       4315
образование      4022
свадьба          2348
Name: purpose_group, dtype: int64

In [247]:
data = data.drop(columns='purpose_lemm') # удалим столбец с леммами, он больше не нужен

data.head()

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_group
0,1,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость
1,1,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль
2,0,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость
3,3,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование
4,0,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба


Чаще всего берут кредиты на различные операции с недвижимостью - почти половина данных в датасете так или иначе связана с покупкой, ремонтом или строительством. Удивительно, что займы на образование почти также распространены, как автокредиты. А вот на свадьбу подают заявки реже всего.

### Категоризация данных

Для дальнейшего анализа и ответов на вопросы о зависимости уровня дохода и наличия детей и возвратом кредита в срок необходимо категоризировать эти данные.

Сведения о детях можно разделить на три категории: 
- бездетные 
- с детьми - 1-2 ребенка
- многодетные - три и более


In [258]:
def children_group(children): # функция, которая распределит по группам согласно условию 
    if children == 0:
        return 'бездетные'
    if children <= 2:
        return 'с детьми'
    return 'многодетные'

data['children_group'] = data['children'].apply(children_group) # добавляем колонку с категорией


data['children_group'].value_counts() # проверяем результат


бездетные      14196
с детьми        6949
многодетные      380
Name: children_group, dtype: int64

Если бы в услових проекта было указано, что все предоставленные данные заемщиков например из Москвы, то можно было бы ориентироваться на уровень зарплат конкретно этого города. Но поскольку город не указан, предположим, что это данные по всей стране, а значит возьмем за основу цифры Росстата, согласно которым среднемесячная зарплата по итогам 2020 года составляет 51000. 

Соответственно, все что ниже этой планки принимается как низкий доход. Все, что больше 200000 - высокий. Отдельная категория - от пятиста тысяч, это заемщики с очень высоким уровнем дохода.

In [259]:
def income_group(total_income):  # функия присваивает категорию согласно условиям
    if total_income <= 51000:
        return 'низкий'
    if total_income <= 200000:
        return 'средний'
    if total_income <= 500000:
        return 'высокий'
    return 'очень высокий'

data['income_group'] = data['total_income'].apply(income_group) # добавляем колонку с категориями

In [260]:
data['income_group'].value_counts() # проверим результат


средний          16050
высокий           4845
низкий             408
очень высокий      222
Name: income_group, dtype: int64

У очень богатых кредиты менее востребованы, как и у тех, чей доход ниже среднего. Возможно последним их просто реже дают. Чаще всего к банку обращаются люди среднего класса с доходом от 50000 до 200000 рублей. 

## Ответы на вопросы

### Есть ли зависимость между наличием детей и возвратом кредита в срок?



In [277]:
# выводим сводную таблицу, где видно кол-во должников и тех, кто платил вовремя
data_pivot_children = data.pivot_table(
    index=['children_group'], columns='debt',
    values='gender', aggfunc='count')
 
# добавляем колонку с % просроченных кредитов        
data_pivot_children['ratio'] = data_pivot_children[1] / (
    data_pivot_children[1] + data_pivot_children[0]) 


display(data_pivot_children)

print('Доля должников среди заемщиков без детей равна {:.1%}'.format(
    data_pivot_children.loc[('бездетные'), 'ratio']))

print('Доля должников среди заемщиков с тремя детьми и более равна {:.1%}'.format(
    data_pivot_children.loc[('многодетные'), 'ratio']))

print('Доля должников среди заемщиков с 1-2 ребенком равна {:.1%}'.format(
    data_pivot_children.loc[('с детьми'), 'ratio']))

                                       

debt,0,1,ratio
children_group,,,
бездетные,13132,1064,0.074951
многодетные,349,31,0.081579
с детьми,6303,646,0.092963


Доля должников среди заемщиков без детей равна 7.5%
Доля должников среди заемщиков с тремя детьми и более равна 8.2%
Доля должников среди заемщиков с 1-2 ребенком равна 9.3%


Бездетные заемщики выглядят самыми дисциплинированными очевидно по причине более низкой финансовой нагрузки, а вот многодетные просрачивают платежи реже, чем те, у кого один или два ребенка.

### Есть ли зависимость между семейным положением и возвратом кредита в срок?



In [292]:
# сводная таблица кол-ва должников 
data_pivot_family = data.pivot_table(
    index=['family_status'], columns='debt',
    values='children', aggfunc='count')

# добавляем колонку с % просроченных кредитов
data_pivot_family['ratio'] = data_pivot_family[1] / (
    data_pivot_family[1] + data_pivot_family[0])

# выводим и сортируем по %
display(data_pivot_family.sort_values(by='ratio'))

print(('Доля должников среди овдовевших равна {:.1%}'.format(
    data_pivot_family.loc[('вдовец / вдова'), 'ratio'])))

print(('Доля должников среди разведенных равна {:.1%}'.format(
    data_pivot_family.loc[('в разводе'), 'ratio'])))

print(('Доля должников среди состоящих в браке равна {:.1%}'.format(
    data_pivot_family.loc[('женат / замужем'), 'ratio'])))

print(('Доля должников среди состоящих в гражданском браке равна {:.1%}'.format(
    data_pivot_family.loc[('гражданский брак'), 'ratio'])))

print(('Доля должников среди не состоящих в браке равна {:.1%}'.format(
    data_pivot_family.loc[('не женат / не замужем'), 'ratio'])))


debt,0,1,ratio
family_status,,,
вдовец / вдова,897,63,0.065625
в разводе,1110,85,0.071130
женат / замужем,11449,931,0.075202
гражданский брак,3789,388,0.092890
не женат / не замужем,2539,274,0.097405


Доля должников среди овдовевших равна 6.6%
Доля должников среди разведенных равна 7.1%
Доля должников среди состоящих в браке равна 7.5%
Доля должников среди состоящих в гражданском браке равна 9.3%
Доля должников среди не состоящих в браке равна 9.7%


Неженатые/не замужние наименее благонадежные заемщики, а также те, кто состоит в гражданском браке. Видимо, в случае гражданского брака сложнее рассчитывать на поддержку партнера в случае необходимости, чем в законном. Вдовствующие и разведенные реже допускают просрочки, однако их количество в датасете сильно меньше остальных, чтобы говорить о заметных различиях.

###  Есть ли зависимость между уровнем дохода и возвратом кредита в срок?



In [295]:
# считаем кол-во должников и платящих вовремя по категориям дохода
data_pivot_income = data.pivot_table(
    index=['income_group'], columns='debt',
    values='children', aggfunc='count')

# добавляем % просрочек
data_pivot_income['ratio'] = data_pivot_income[1] / (
    data_pivot_income[1] + data_pivot_income[0]) 

# выводим результат с сортировкой по % просрочек
display(data_pivot_income.sort_values(by='ratio'))

print(('Доля должников среди заемщиков с доходом от 500 000 равна {:.1%}'.format(
    data_pivot_income.loc[('очень высокий'), 'ratio'])))

print(('Доля должников среди заемщиков с доходом до 51 000 равна {:.1%}'.format(
    data_pivot_income.loc[('низкий'), 'ratio'])))

print(('Доля должников среди заемщиков с доходом от 200 000 до 500 000 равна {:.1%}'.format(
    data_pivot_income.loc[('высокий'), 'ratio'])))

print(('Доля должников среди заемщиков с доходом от 51 000 до 200 000 равна {:.1%}'.format(
    data_pivot_income.loc[('средний'), 'ratio'])))

debt,0,1,ratio
income_group,,,
очень высокий,208,14,0.063063
низкий,382,26,0.063725
высокий,4501,344,0.071001
средний,14693,1357,0.084548


Доля должников среди заемщиков с доходом от 500 000 равна 6.3%
Доля должников среди заемщиков с доходом до 51 000 равна 6.4%
Доля должников среди заемщиков с доходом от 200 000 до 500 000 равна 7.1%
Доля должников среди заемщиков с доходом от 51 000 до 200 000 равна 8.5%


В целом различия между группами не такие большие, чтобы говорить о заметной тенденции. Люди с очень высоким доходом реже всего допускают просрочки, но при этом и кредиты для них не так актуальны - их заметно меньше, чем заемщиков со средним или выше среднего доходами. В категории "низкий доход" также небольшой % должников, однако их тоже меньше, вероятно по причине более жестких требований кредиты им одобряют значительно реже или ставят менее выгодные условия. В группе риска самый распространенный средний класс, клиенты с доходом от 51000 до 200000 чаще всего не следят за графиком платежей.


### Как разные цели кредита влияют на его возврат в срок?

In [298]:
# сводная таблица в разбивке по целям 
data_pivot_purpose = data.pivot_table(
    index=['purpose_group'], columns='debt',
    values='children', aggfunc='count')

# добавляем % должников
data_pivot_purpose['ratio'] = data_pivot_purpose[1] / (
    data_pivot_purpose[1] + data_pivot_purpose[0]) 

# выводим результат с сортировкой по %
display(data_pivot_purpose.sort_values(by='ratio'))

print(('Процент должников среди кредитов на недвижимость, строительство и ремонт - {:.1%}'.format(
    data_pivot_purpose.loc[('недвижимость'), 'ratio'])))

print(('Процент должников среди кредитов на свадьбу - {:.1%}'.format(
    data_pivot_purpose.loc[('свадьба'), 'ratio'])))

print(('Процент должников среди кредитов на образование - {:.1%}'.format(
    data_pivot_purpose.loc[('образование'), 'ratio'])))

print(('Процент должников среди автокредитов - {:.1%}'.format(
    data_pivot_purpose.loc[('автомобиль'), 'ratio'])))


debt,0,1,ratio
purpose_group,,,
недвижимость,10058,782,0.072140
свадьба,2162,186,0.079216
образование,3652,370,0.091994
автомобиль,3912,403,0.093395


Процент должников среди кредитов на недвижимость, строительство и ремонт - 7.2%
Процент должников среди кредитов на свадьбу - 7.9%
Процент должников среди кредитов на образование - 9.2%
Процент должников среди автокредитов - 9.3%


Кредиты на приобретение жилья, строительство и ремонт чаще других выплачиваются вовремя. Однако у нас здесь нет информации, это в том числе ипотечные кредиты или потребительски с указанием цели на изменение жилищных условий. Было бы интересно также посмотреть отдельно на коммерческую недвижимость - такая же там статистика как по жилой, или отличается. Удивительно, что доля просроченных кредитов на свадьбу всего около 8% - вероятно супруги гасят его уже совместными усилиями, либо закрывают досрочно подаренными по случаю свадьбы деньгами. Автокредиты и кредиты на образование чаще других просрачиваются плательщиками.

## Общий вывод

Самыми неблагонадежными заемщиками выглядят люди, не состоящие в браке и имеющие 1-2 детей, это предсказуемый результат, так как их финансовая нагрузка в случае наличия детей выше и при этом нет помощи от партнера/она незначительная. Более выгодным клиентом являются бездетные, но при этом состоящие в официальном браке и оформляющие кредит на покупку/ремонт/строительство недвижимости.

Многодетные и люди с низким доходом вопреки ожиданиям не входят в группу риска и реже других допускают просрочки платежей -  к таким заемщикам банки устанавливают более высокие требования, более строго проверяют перед одобрением кредита. Возможно поэтому они более дисциплинированные, так как в их случае просрочка может обернуться существенными для их бюджета процентами.

То, что кредиты на недвижимость чаще других выплачиваются вовремя, выглядит достаточно правдоподобно, так как они менее импульсивные - люди обычно тщательно просчитывают будущие траты, прежде чем ввязываться в операции с недвижимостью. 
А вот результат с кредитами на свадьбу удивил, ведь известно, что в течение первых двух лет в России распадается каждый второй брак, а значит, что в случае оформления кредита и последующего развода обязательство по его выплате скорее всего ложится на плечи только одного из партнеров. Но видимо, этот тип кредитов успевают погасить и платят более исправно.

Однако разница должников в разных группах колеблется в пределах 1%-3%, что вряд ли позволяет записать определенную категорию людей в гарантированно ненадежные или наоборот, в потенциально стабильные и заслуживающие доверия. А вот совпадение нескольких факторов уже делает способность потенциального заёмщика вернуть/просрочить кредит банку более предсказуемой.

Для более показательных результатов было бы здорово иметь более детальные данные о просрочках - это единичный случай или повторяющееся явление. Одна просрочка платежа в кредитной истории не делает заемщика злостным неплательщиком, так как могла произойти вследствие например невнимательности и в дальнейшем не повториться. А вот регулярные задержки уже бы указывали на тенденцию и позволили бы дать более точный прогноз.